<a href="https://colab.research.google.com/github/DataJenius/NLPEncodingExperiment/blob/main/python/NLPEncodingExperiment_get_BERT_ft_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
###########################################
# install all dependencies
!pip install transformers
!pip install datasets

     |████████████████████████████████| 3.8 MB 16.8 MB/s 
     |████████████████████████████████| 596 kB 62.9 MB/s 
     |████████████████████████████████| 67 kB 6.3 MB/s 
     |████████████████████████████████| 895 kB 58.9 MB/s 
     |████████████████████████████████| 6.5 MB 41.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 312 kB 23.3 MB/s 
     |████████████████████████████████| 1.1 MB 66.7 MB/s 
     |████████████████████████████████| 134 kB 63.8 MB/s 
     |████████████████████████████████| 212 kB 72.7 MB/s 
     |████████████████████████████████| 127 kB 73.6 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
     |████████████████████████████████| 144 kB 74.1 MB/s 
     |████████████████████████████████| 271 kB 70.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urll

In [ ]:
###########################################
# load dependencies
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, Split
import pandas as pd
import numpy as np
import os
import torch
from google.colab import files

In [ ]:
###############################################
# use BERT tokenizer & classifier
auto_cls = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
auto_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')


# define our tokenizer params here
# use max_length padding with 500 tokens
def tokenize(row):
    return auto_tokenizer(row['raw_text'], 
                          truncation=True, 
                          padding='max_length',
                          max_length=500)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
#################################################################################
# our labelled, raw comment data is on github
# already broken into 5 validation groups for the sake of 5 models
# use 20% of data for val, 20% for test, and keep 60% for training
df_links_to_data = pd.DataFrame({'my_group': [1,2,3,4,5], 
                                 'link': ['https://raw.githubusercontent.com/DataJenius/NLPEncodingExperiment/main/data/comments/selected/selected_reddit_comments_group1.csv',
                                          'https://raw.githubusercontent.com/DataJenius/NLPEncodingExperiment/main/data/comments/selected/selected_reddit_comments_group2.csv',
                                          'https://raw.githubusercontent.com/DataJenius/NLPEncodingExperiment/main/data/comments/selected/selected_reddit_comments_group3.csv',
                                          'https://raw.githubusercontent.com/DataJenius/NLPEncodingExperiment/main/data/comments/selected/selected_reddit_comments_group4.csv',
                                          'https://raw.githubusercontent.com/DataJenius/NLPEncodingExperiment/main/data/comments/selected/selected_reddit_comments_group5.csv']}) 

In [ ]:
######################################################################################
# our data is already split into 5 groups, fully labelled

# setup this experiment
test_input_method = 'BERT (768)'

# split into val/test/train based on validation group 
df_train = df_links_to_data.loc[df_links_to_data['my_group'].isin([1,2,3])]
df_val = df_links_to_data.loc[df_links_to_data['my_group'] == 4]
df_test = df_links_to_data.loc[df_links_to_data['my_group'] == 5]


# load our train dataset (60%) 
train_dataset = load_dataset(path='csv', 
                             data_files=df_train['link'].values.tolist(),
                             split=Split.ALL).rename_column("label", "labels").remove_columns(['msg_id','token_count','my_group','my_role'])
print("\n\ntrain - group ",df_train['my_group'].unique())
print(train_dataset) 

# load our validation dataset (20%) 
val_dataset = load_dataset(path='csv', 
                           data_files=df_val['link'].values.tolist(),
                           split=Split.ALL).rename_column("label", "labels").remove_columns(['msg_id','token_count','my_group','my_role'])
print("\n\nval - group ",df_val['my_group'].unique())
print(val_dataset)   

# load our test dataset (20%) 
test_dataset = load_dataset(path='csv', 
                            data_files=df_test['link'].values.tolist(),
                            split=Split.ALL).rename_column("label", "labels")
print("\n\ntest - group ",df_test['my_group'].unique())
print(test_dataset)   


# tokenize the datasets
tokenized_train_dataset = train_dataset.map(tokenize, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize, batched=True)

Using custom data configuration default-d560abb8ebad5240


  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/501k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/536k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/533k [00:00<?, ?B/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-cba455d41c7e57e4/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


test - group  [5]
Dataset({
    features: ['msg_id', 'token_count', 'my_group', 'my_role', 'labels', 'source', 'raw_text', 'clean_text'],
    num_rows: 2000
})


  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
#################################################################################
# define our trainer and fine-tune BERT

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# takes about 4hrs for 1 epoch
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# set our training arguments
training_args = TrainingArguments(
    output_dir='output',
    num_train_epochs=10,             
    per_device_train_batch_size=1,
    per_device_eval_batch_size=8,
    evaluation_strategy='steps',
    eval_steps=250,
    logging_steps=250,
    gradient_accumulation_steps=8,
)

# functon to compute our metrics
def compute_metrics(eval_pred):
    predictions = eval_pred.predictions
    labels = eval_pred.label_ids
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

# define our trainer
trainer = Trainer(model=auto_cls, 
                  args=training_args, 
                  train_dataset=tokenized_train_dataset, 
                  eval_dataset=tokenized_val_dataset,
                  compute_metrics=compute_metrics)    

# make the magic happen
trainer.train()
#trainer.evaluate()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: raw_text, clean_text, source. If raw_text, clean_text, source are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 6000
  Num Epochs = 10
  Instantaneous batch 

Step,Training Loss,Validation Loss,Accuracy
250,0.006100,0.370641,0.968500
500,0.135400,0.216723,0.956000
750,0.199000,0.248574,0.936500
1000,0.154500,0.176738,0.966500
1250,0.112700,0.217040,0.959000
1500,0.110700,0.211595,0.962000
1750,0.079200,0.251239,0.952500
2000,0.082300,0.198303,0.964000


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: raw_text, clean_text, source. If raw_text, clean_text, source are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: raw_text, clean_text, source. If raw_text, clean_text, source are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8
Saving model checkpoint to output/checkpoint-500
Configuration saved in output/checkpoint-500/config.json
Model weights saved in output/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argum

In [ ]:
###############################################
# save the model 
#folder_name = 'BERT_modelv1' # fine-tuned 1 epoch
folder_name = 'BERT_modelv10' # fine-tuned 10 epochs
print(folder_name)
trainer.save_model(folder_name)
os.listdir(folder_name)

In [ ]:
############################################################################
# load our model #1
#folder_name = 'BERT_modelv1' # fine-tuned 1 epoch
folder_name = 'BERT_modelv10' # fine-tuned 10 epochs
config = AutoModelForSequenceClassification.from_pretrained(folder_name, output_hidden_states=True)
loaded_model = AutoModelForSequenceClassification.from_pretrained(folder_name, config=config)
print(loaded_model.device)

# get details about the model
def count_all_parameters(model):
    return sum(p.numel() for p in model.parameters())
def count_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
num_params = count_all_parameters(loaded_model)
num_params_train = count_trainable_parameters(loaded_model)
print(loaded_model)
print(num_params, "total params")
print(num_params_train, "trainable params")

loading configuration file BERT_modelv1/config.json
Model config BertConfig {
  "_name_or_path": "BERT_modelv1",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file BERT_modelv1/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSe

cpu
BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, ele

In [ ]:
#################################################################################
# load our full dataset (100%) 
full_dataset = load_dataset(path='csv', 
                            data_files=df_links_to_data['link'].values.tolist(),
                            split=Split.ALL).rename_column("label", "labels")

#tokenize                          
tokenized_full_dataset = full_dataset.map(tokenize, batched=True)     

# turn tokenized test data into tensors
input_ids = torch.tensor(tokenized_full_dataset['input_ids'])
attention_mask = torch.tensor(tokenized_full_dataset['attention_mask'])
print(input_ids.shape)
print(attention_mask.shape)

Using custom data configuration default-71ca0bbdb5b5865a
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-71ca0bbdb5b5865a/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-71ca0bbdb5b5865a/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-3eb533fd8df890ab.arrow


torch.Size([10000, 500])
torch.Size([10000, 500])


In [ ]:
#################################################################################
# forgive this goofy loop, I wanted to be able to sanity-check at each step
# loop through all 10,000 comments
# takes about 46 min to get fine-tuned BERT embeddings from them all
loaded_model.eval()
df_embeddings = pd.DataFrame()
j = 0
for i in range(0,10000,1):
  j += 1
  print(i, j)

  # this specific input as tensors
  my_input_ids = input_ids[i:i+1]
  my_attention_mask = attention_mask[i:i+1]

  # predict class for this comment
  my_output = loaded_model(input_ids=my_input_ids,
                        attention_mask=my_attention_mask,
                        output_hidden_states=True)

  # get the final 768-dimension [CLS] embedding for the comment according to BERT
  my_hidden_states = torch.stack(list(my_output["hidden_states"]), dim=0)
  my_cls = my_hidden_states[12][0][0]
  
  # put it into a dataframe for use elsewhere
  values = my_cls.squeeze().tolist()
  df_my_embeddings = pd.DataFrame({'value': values}).transpose()
  df_my_embeddings["msg_id"] = full_dataset["msg_id"][i]
  df_my_embeddings.index=[i]
  df_embeddings = df_embeddings.append(df_my_embeddings) 

  # save every 1000 records
  if j == 1000:

      # save results to local CSV
      file_name = 'BERT_ft_embeddings_i'+str(i)+'.csv'
      df_embeddings.to_csv(file_name, index=False) 
      files.download(file_name)
      print("\n\nsave progress...\n\n",df_embeddings.head(10))      

      # reset df and j counter
      df_embeddings = pd.DataFrame()
      j=0

  #if i == 1:
  #  break    


0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 10
10 11
11 12
12 13
13 14
14 15
15 16
16 17
17 18
18 19
19 20
20 21
21 22
22 23
23 24
24 25
25 26
26 27
27 28
28 29
29 30
30 31
31 32
32 33
33 34
34 35
35 36
36 37
37 38
38 39
39 40
40 41
41 42
42 43
43 44
44 45
45 46
46 47
47 48
48 49
49 50
50 51
51 52
52 53
53 54
54 55
55 56
56 57
57 58
58 59
59 60
60 61
61 62
62 63
63 64
64 65
65 66
66 67
67 68
68 69
69 70
70 71
71 72
72 73
73 74
74 75
75 76
76 77
77 78
78 79
79 80
80 81
81 82
82 83
83 84
84 85
85 86
86 87
87 88
88 89
89 90
90 91
91 92
92 93
93 94
94 95
95 96
96 97
97 98
98 99
99 100
100 101
101 102
102 103
103 104
104 105
105 106
106 107
107 108
108 109
109 110
110 111
111 112
112 113
113 114
114 115
115 116
116 117
117 118
118 119
119 120
120 121
121 122
122 123
123 124
124 125
125 126
126 127
127 128
128 129
129 130
130 131
131 132
132 133
133 134
134 135
135 136
136 137
137 138
138 139
139 140
140 141
141 142
142 143
143 144
144 145
145 146
146 147
147 148
148 149
149 150
150 151
151 152
15

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>



save progress...

           0         1         2         3         4         5         6  \
0  0.047403 -0.051700 -0.365815  0.148061 -0.535689  0.223934  1.254592   
1 -0.009199 -0.072623 -0.357462  0.161104 -0.554575  0.178139  1.274894   
2 -0.075791 -0.158615  0.679475  0.216228  0.701845 -0.268213 -0.595190   
3 -0.041176 -0.171742  0.641455  0.197139  0.736751 -0.294790 -0.594111   
4  0.007319 -0.064146 -0.330732  0.163415 -0.545964  0.166491  1.268156   
5 -0.456374  0.029407  0.329871  0.465754  0.065535 -0.200851  0.852477   
6 -0.153701 -0.423255  0.115487  0.261997  0.803537 -0.122127 -0.539995   
7  0.024753 -0.065737 -0.338129  0.156204 -0.546920  0.186542  1.250067   
8 -0.095647  0.008245 -0.031380  0.152259 -0.463306  0.042561  1.305223   
9  0.021871 -0.078406 -0.310470  0.139376 -0.572395  0.181374  1.276895   

          7         8         9  ...       759       760       761       762  \
0 -0.656984  0.095141  0.663508  ...  0.769508 -0.696395  0.017928 -0.924

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>



save progress...

              0         1         2         3         4         5         6  \
8000  0.057927 -0.287445  0.342658  0.392008  0.565954  0.043023 -0.647841   
8001 -0.048859 -0.163370  0.622231  0.177224  0.663643 -0.275797 -0.553731   
8002  0.062954 -0.064414 -0.364925  0.137911 -0.569051  0.233845  1.257622   
8003 -0.145357 -0.251255  0.637669  0.234555  0.707810 -0.330420 -0.578522   
8004 -0.005872 -0.075093 -0.315126  0.192450 -0.557971  0.157828  1.260165   
8005 -0.027792 -0.465784  0.389712  0.285481  0.547823 -0.221042 -0.640480   
8006  0.050902 -0.061339 -0.365108  0.138145 -0.569240  0.224259  1.258135   
8007 -0.494742 -0.334001  0.492801  0.011450 -0.040505 -0.131940  0.724362   
8008  0.015104 -0.061483 -0.342582  0.134308 -0.578252  0.184926  1.258627   
8009 -0.114479 -0.433461  0.581285  0.246159  0.629216 -0.342956 -0.517615   

             7         8         9  ...       759       760       761  \
8000 -0.253957 -0.006036 -0.580096  ... -0.4878

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>



save progress...

              0         1         2         3         4         5         6  \
9000  0.055796 -0.049115 -0.363718  0.168785 -0.573163  0.209320  1.257403   
9001 -0.047628 -0.089915  0.647658  0.241972  0.730335 -0.260990 -0.548218   
9002 -0.023620 -0.095473  0.595040  0.186929  0.702979 -0.305454 -0.574254   
9003  0.035860 -0.076302 -0.365307  0.123614 -0.552568  0.220753  1.268197   
9004 -0.018650 -0.067399 -0.323692  0.164673 -0.570434  0.170183  1.274333   
9005 -0.166006 -0.143967  0.229506  0.203448  0.025793  0.009663  0.540498   
9006 -0.098709 -0.075307  0.706803  0.213221  0.716305 -0.259522 -0.543374   
9007 -0.100227 -0.300119  0.134920  0.262598  0.679394 -0.046251 -0.546484   
9008 -0.066851 -0.127621  0.663375  0.194584  0.641335 -0.286178 -0.551678   
9009 -0.068170 -0.512117  0.258940  0.376254  0.683081 -0.128783 -0.536922   

             7         8         9  ...       759       760       761  \
9000 -0.656502  0.092077  0.686333  ...  0.7579

In [ ]:
print(df_embeddings.head)

<bound method NDFrame.head of           0         1         2         3         4         5         6  \
0  0.047403 -0.051700 -0.365815  0.148061 -0.535689  0.223934  1.254592   
1 -0.009199 -0.072623 -0.357462  0.161104 -0.554575  0.178139  1.274894   

          7         8         9  ...       759       760       761       762  \
0 -0.656984  0.095141  0.663508  ...  0.769508 -0.696395  0.017928 -0.924579   
1 -0.630651  0.109574  0.713033  ...  0.785477 -0.747074 -0.003504 -0.898637   

        763       764       765       766       767  msg_id  
0 -0.223866 -0.158940  0.204260  1.276494 -0.904730   11702  
1 -0.197978 -0.183148  0.178438  1.259046 -0.945903    3854  

[2 rows x 769 columns]>


In [ ]:
############################################################################
#string_to_check = 'I love Luke Skywalker, but I hate Gandalf the Grey.'
#string_to_check = 'I love Gandalf the Grey, but I hate Luke Skywalker.'
string_to_check = 'Captain Smurgleblorp snorged his 😜 on Blursday.'
print(string_to_check)

# encode into IDs for each token
encoded = auto_tokenizer.encode(string_to_check)
print(encoded)

# convert IDs into their tokens
tokens = auto_tokenizer.convert_ids_to_tokens(encoded)
print(tokens)

# convert back into a sentence
decoded = auto_tokenizer.decode(encoded)
print(decoded)

Captain Smurgleblorp snorged his 😜 on Blursday.
[101, 2952, 15488, 12514, 2571, 16558, 2953, 2361, 1055, 12131, 5999, 2010, 100, 2006, 14819, 16150, 4710, 1012, 102]
['[CLS]', 'captain', 'sm', '##urg', '##le', '##bl', '##or', '##p', 's', '##nor', '##ged', 'his', '[UNK]', 'on', 'blur', '##sd', '##ay', '.', '[SEP]']
[CLS] captain smurgleblorp snorged his [UNK] on blursday. [SEP]
